In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c cscs460-project

cscs460-project.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/cscs460-project.zip

Archive:  /content/cscs460-project.zip
replace imageset/test/0.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [ ]:
train_df = pd.read_csv("/content/train.csv")
print(train_df)

        id                                           path  label
0        0  imageset/train/0/ILSVRC2012_val_00000293.JPEG      0
1        1  imageset/train/0/ILSVRC2012_val_00002138.JPEG      0
2        2  imageset/train/0/ILSVRC2012_val_00003014.JPEG      0
3        3  imageset/train/0/ILSVRC2012_val_00006697.JPEG      0
4        4  imageset/train/0/ILSVRC2012_val_00007197.JPEG      0
...    ...                                            ...    ...
9464  9464           imageset/train/9/n03445777_9947.JPEG      9
9465  9465           imageset/train/9/n03445777_9966.JPEG      9
9466  9466           imageset/train/9/n03445777_9976.JPEG      9
9467  9467            imageset/train/9/n03445777_998.JPEG      9
9468  9468           imageset/train/9/n03445777_9993.JPEG      9

[9469 rows x 3 columns]


In [ ]:
# Split the dataset into training set and validation set, stratified by the labels
train_images, val_images, train_labels, val_labels = train_test_split(
    train_df['path'], train_df['label'], test_size=0.2, random_state=42, stratify=train_df['label'])

# Print the sizes of the training set and validation set
print(f"Training set size: {len(train_images)}")
print(f"Validation set size: {len(val_images)}")

Training set size: 7575
Validation set size: 1894


In [ ]:
print(train_images)

512      imageset/train/0/n01440764_3198.JPEG
4116    imageset/train/4/n03000684_15363.JPEG
5048    imageset/train/5/n03028079_20538.JPEG
8039    imageset/train/8/n03425413_18325.JPEG
9310     imageset/train/9/n03445777_7756.JPEG
                        ...                  
5104    imageset/train/5/n03028079_24636.JPEG
8136    imageset/train/8/n03425413_20664.JPEG
7314     imageset/train/7/n03417042_5153.JPEG
4347    imageset/train/4/n03000684_24885.JPEG
292     imageset/train/0/n01440764_14294.JPEG
Name: path, Length: 7575, dtype: object


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the target size for resizing the images
target_size = (224, 224)
batch_size = 16

# Create an ImageDataGenerator for the training set
train_datagen = ImageDataGenerator(rescale=1./255)

# Create an ImageDataGenerator for the validation set
val_datagen = ImageDataGenerator(rescale=1./255)

# Create a DataFrame with the image paths and labels for the training set
train_df_images = pd.DataFrame({'path': train_images, 'label': train_labels.astype(str)})

# Create a DataFrame with the image paths and labels for the validation set
val_df_images = pd.DataFrame({'path': val_images, 'label': val_labels.astype(str)})

# Create a generator for the training set
train_generator = train_datagen.flow_from_dataframe(
    train_df_images,
    x_col='path',
    y_col='label',
    target_size=target_size,
    class_mode='sparse',
    batch_size=batch_size
)

# Create a generator for the validation set
val_generator = val_datagen.flow_from_dataframe(
    val_df_images,
    x_col='path',
    y_col='label',
    target_size=target_size,
    class_mode='sparse',
    batch_size=batch_size
)

Found 7575 validated image filenames belonging to 10 classes.
Found 1894 validated image filenames belonging to 10 classes.


In [ ]:
train_generator

In [ ]:
# # Print the sizes of the resized training set and validation set
# print(f"Resized training set size: {len(train_images_resized)}")
# print(f"Resized validation set size: {len(val_images_resized)}")

Resized training set size: 7575
Resized validation set size: 1894


In [ ]:
base = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

In [ ]:
base.trainable = False

In [ ]:
base.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [ ]:
dense1 = layers.Dense(512, activation = "relu")
flatten1 = layers.Flatten()
dense2 = layers.Dense(256, activation = "relu")
dense3 = layers.Dense(10, activation = "softmax")

In [ ]:
x = dense1(base.output)
x = flatten1(x)

In [ ]:
y = dense2(x)

In [ ]:
z = dense3(y)

In [ ]:
model = keras.Model(inputs = base.input, outputs = z)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [ ]:
# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/model_result1/',  # Filepath to save the model
    monitor='val_loss',  # Metric to monitor (val loss in this case)
    save_best_only=True,  # Save only the best model
    save_weights_only=False,  # Save the entire model (including architecture)
    mode='min'  # Mode for comparison (minimizing the monitored metric)
)

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (val loss in this case)
    patience=5,  # Number of epochs with no improvement after which training will be stopped
    mode='min'  # Mode for comparison (minimizing the monitored metric)
)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
epochs = 50
batch_size = 8

history = model.fit(train_generator,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=val_generator,
                    callbacks=[checkpoint_callback, early_stopping_callback])

Epoch 1/50
474/474 [==============================] - ETA: 0s - loss: 0.1920 - accuracy: 0.9572

474/474 [==============================] - 122s 230ms/step - loss: 0.1920 - accuracy: 0.9572 - val_loss: 0.1361 - val_accuracy: 0.9604
Epoch 2/50
474/474 [==============================] - ETA: 0s - loss: 0.0781 - accuracy: 0.9822

474/474 [==============================] - 94s 199ms/step - loss: 0.0781 - accuracy: 0.9822 - val_loss: 0.0854 - val_accuracy: 0.9810
Epoch 3/50
474/474 [==============================] - 83s 175ms/step - loss: 0.0798 - accuracy: 0.9838 - val_loss: 0.1132 - val_accuracy: 0.9757
Epoch 4/50
474/474 [==============================] - 78s 166ms/step - loss: 0.0708 - accuracy: 0.9875 - val_loss: 0.1119 - val_accuracy: 0.9789
Epoch 5/50
474/474 [==============================] - 78s 164ms/step - loss: 0.0316 - accuracy: 0.9913 - val_loss: 0.2120 - val_accuracy: 0.9778
Epoch 6/50
474/474 [==============================] - 84s 177ms/step - loss: 0.0259 - accuracy: 0.9956 - val_loss: 0.1397 - val_accuracy: 0.9725
Epoch 7/50
474/474 [==============================] - 77s 163ms/step - loss: 0.0349 - accuracy: 0.9942 - val_loss: 0.1288 - val_accuracy: 0.9873
Epoch 8/50
474/474 [==============================] - 78s 164ms/step - loss: 0.0464 - accuracy: 0.9925 - val_loss: 0.2289 - val_accuracy: 0.9

In [ ]:
# Load and preprocess the test images
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("/content/test.csv")
test_images = test_df['path']
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col=None,
    target_size=target_size,
    class_mode=None,
    batch_size=batch_size,
    shuffle=False
)


Found 3925 validated image filenames.


In [ ]:
# Make predictions on the test dataset
predictions = model.predict(test_generator)

491/491 [==============================] - 34s 69ms/step


In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
print(predicted_labels)

[0 0 0 ... 9 9 9]


In [ ]:
results_df = pd.DataFrame({'label': predicted_labels})

In [ ]:
results_df.tail(20)

,label
3905,9
3906,9
3907,9
3908,9
3909,9
3910,9
3911,9
3912,9
3913,9
3914,9


In [ ]:
results_df.to_csv('/content/predictions.csv')